In [1]:
import scipy.sparse as sp
import pandas as pd
import numpy as np


class Dataset(object):
    def __init__(self, path):
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape

    def load_rating_file_as_list(self, filename):
        df = pd.read_csv(filename, sep="\t")
        ratingList = list(zip(df.userid.tolist(), df.itemid.tolist()))
        return ratingList
    
    def load_negative_file(self, filename):
        df = pd.read_csv(filename, sep="\t")
        negativeList = df.iloc[:, 1:].values.tolist()
        return negativeList

    def load_rating_file_as_matrix(self, filename):
        df = pd.read_csv(filename, sep="\t")
        num_users = df.userid.max()
        num_items = df.itemid.max()
        mat = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
        interactions = df[['userid', 'itemid']].values.tolist()
        # [(0, 2969), (0, 1178), (0, 1574), (0, 957)]
        for user, item in interactions:
            mat[user, item] = 1.
        # [((0, 2969), 1.0), ((0, 1178), 1.0), ((0, 1574), 1.0), ((0, 957), 1.0)]
        return mat

In [2]:
import math
import heapq
import numpy as np


def evaluate_model(model, testRatings, testNegatives, topK):

    global _model
    global _testRatings
    global _testNegatives
    global _topK

    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _topK = topK

    hits, ndcgs = [],[]
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return (hits, ndcgs)


def eval_one_rating(idx):

    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)],
                                 batch_size=100, verbose=0)

    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()

    ranklist = heapq.nlargest(_topK, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)


def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

def get_train_instances(train, n_items, n_neg, testNegatives):
    user, item, labels = [],[],[]
    n_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user.append(u)
        item.append(i)
        labels.append(1)
        # negative instances: we also need to make sure they are not in the
        # test dataset
        for t in range(n_neg):
            j = np.random.randint(n_items)
            while ((u, j) in train.keys()) or (j in testNegatives[u]):
                j = np.random.randint(n_items)
            user.append(u)
            item.append(j)
            labels.append(0)
    return np.array(user), np.array(item), np.array(labels)

In [3]:
import numpy as np
import pandas as pd
import os
import heapq
import tensorflow.keras
from tensorflow.keras import initializers
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Dense, Embedding, Input, Dropout, Flatten, concatenate
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2

from time import time

def MLP(n_users, n_items, layers, dropouts, reg):
    num_layer = len(layers)  # MLP 层数

    user = Input(shape=(1,), dtype='int32', name='user_input')
    item = Input(shape=(1,), dtype='int32', name='item_input')

    # user 和 item embedding
    # [?, 1, 32]
    MLP_Embedding_User = Embedding(
        input_dim=n_users,
        output_dim=int(layers[0] / 2),
        name='user_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)
    # [?, 1, 32]
    MLP_Embedding_Item = Embedding(
        input_dim=n_items,
        output_dim=int(layers[0] / 2),
        name='item_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)
    
    # [?, 32]
    user_latent = Flatten()(MLP_Embedding_User(user))
    # [?, 32]
    item_latent = Flatten()(MLP_Embedding_Item(item))
    # [?, 64]
    vector = concatenate([user_latent, item_latent])
    
    # vector: [batch_size, embedding_size] -> [?, 64]
    # MLP层
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], activation="relu", kernel_regularizer=l2(reg), name="layer{}".format(idx))
        vector = layer(vector)
        vector = Dropout(dropouts[idx - 1])(vector)
    # vector: [?, 8]
    # [?, 1]
    # 预测层
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name='prediction')(vector)

    model = Model(inputs=[user, item], outputs=prediction)

    return model


if __name__ == '__main__':
    datadir = "Data_Javier/"
    dataname = "ml-1m"
    modeldir = "models"
    layers = [64,32,16,8]
    ll = str(layers[-1])  # last layer
    dropouts = [0,0,0]
    dp = "wdp" if dropouts[0] != 0 else "wodp"
    n_emb = int(layers[0] / 2)
    reg = 0.0
    batch_size = 256
    epochs = 20
    learner = "adam"
    lr = 0.01
    validate_every = 1
    save_model = True
    topK = 10
    n_neg = 4

    modelfname = "keras_MLP" + \
                 "_".join(["_bs", str(batch_size)]) + \
                 "_".join(["_reg", str(reg).replace(".", "")]) + \
                 "_".join(["_lr", str(lr).replace(".", "")]) + \
                 "_".join(["_n_emb", str(n_emb)]) + \
                 "_".join(["_ll", ll]) + \
                 "_".join(["_dp", dp]) + \
                 ".h5"
    modelpath = os.path.join(modeldir, modelfname)
    resultsdfpath = os.path.join(modeldir, 'results_df.p')

    # Loading data
    dataset = Dataset(os.path.join(datadir, dataname))
    train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
    n_users, n_items = train.shape

    # Build model
    model = MLP(n_users, n_items, layers, dropouts, reg)
    if learner.lower() == "adagrad":
        model.compile(optimizer=Adagrad(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "rmsprop":
        model.compile(optimizer=RMSprop(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "adam":
        model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy')
    else:
        model.compile(optimizer=SGD(lr=lr), loss='binary_crossentropy')

    best_hr, best_ndcg, best_iter = 0, 0, 0
    for epoch in range(1, epochs + 1):
        t1 = time()
        user, item, labels = get_train_instances(train, n_items, n_neg, testNegatives)
        hist = model.fit([user, item], labels, batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
        t2 = time()
        if epoch % validate_every == 0:
            (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK)
            hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
            print("Iteration {}: {:.2f}s, HR = {:.4f}, NDCG = {:.4f}, loss = {:.4f}, validated in {:.2f}s"
                  .format(epoch, t2 - t1, hr, ndcg, loss, time() - t2))
            if hr > best_hr:
                best_hr, best_ndcg, best_iter, train_time = hr, ndcg, epoch, t2 - t1
                if save_model:
                    model.save_weights(modelpath, overwrite=True)

    print("End. Best Iteration {}:  HR = {:.4f}, NDCG = {:.4f}. "
          .format(best_iter, best_hr, best_ndcg))
    if save_model:
        print("The best MLP model is saved to {}".format(modelpath))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Iteration 1: 120.57s, HR = 0.5268, NDCG = 0.2879, loss = 0.3485, validated in 24.10s
Iteration 2: 123.29s, HR = 0.5710, NDCG = 0.3183, loss = 0.3111, validated in 15.55s
Iteration 3: 124.44s, HR = 0.6033, NDCG = 0.3423, loss = 0.2919, validated in 21.47s
Iteration 4: 126.90s, HR = 0.6106, NDCG = 0.3488, loss = 0.2822, validated in 15.18s
Iteration 5: 124.31s, HR = 0.6076, NDCG = 0.3522, loss = 0.2757, validated in 15.42s
Iteration 6: 119.69s, HR = 0.6409, NDCG = 0.3696, loss = 0.2712, validated in 22.72s
Iteration 7: 112.47s, HR = 0.6338, NDCG = 0.3630, loss = 0.2672, validated in 22.41s
Iteration 8: 118.33s, HR = 0.6361, NDCG = 0.3664, loss = 0.2640, validated 